# Time Series Forecasting with LSTM - Pytorch

- Since I intend to predict both the Sales Value and Product Name, given the customer name and city, we will develop a model using deep learning techniques, not limited to `LSTM` but I am experimenting that first.
- The steps included in this notebook are for experimenting before choosing the model to add to the live trainng pipeline for production.

I am Glad🤓💻

**Important Notes**

- Predicting the Sales Value is a Linear function, thus I will utilize `nn.Linear` class to create it's linear model
- Predicting the **Product Name** is a classification problem, thus we will have to assign all classes a probability as we predict them. (`Softmax` is a good fit)

In [8]:
import torch
import torchmetrics
import torch.nn as nn
import torch.nn.init as init
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import pandas as pd
import numpy as np
import os
import random
import time
import warnings
from pathlib import Path
from typing import Tuple

warnings.filterwarnings("ignore")

In [2]:
# TODO: Load the train and validation data
data_path = (Path.cwd() / "../data/processed").resolve()
print(f"Loading data from {data_path}")
train_file = data_path / "train.csv"
val_file = data_path / "val.csv"

if not train_file.exists() or not val_file.exists():
	print("Data files not found.")
else:
	train_data = pd.read_csv(train_file)
	val_data = pd.read_csv(val_file)

Loading data from /home/dan/Desktop/time_series_analysis/data/processed


In [3]:
train_data.head()

,Sub-Category,Product Name,dayofweek_cos,Customer Name,State,City,month_cos,Sales
0,5,790,1.000000,380,40,271,-8.660254e-01,1.140088
1,12,1720,0.623490,645,3,415,-5.000000e-01,-0.129866
2,3,1589,0.623490,357,11,388,1.000000e+00,-0.394460
3,10,210,0.623490,700,28,346,-1.836970e-16,-0.362179
4,13,1135,-0.900969,686,20,115,-5.000000e-01,0.551033


In [4]:
train_data = train_data[["Sub-Category", "Customer Name", "State", "City", "dayofweek_cos", "month_cos","Product Name", "Sales"]]
train_data.head()

,Sub-Category,Customer Name,State,City,dayofweek_cos,month_cos,Product Name,Sales
0,5,380,40,271,1.000000,-8.660254e-01,790,1.140088
1,12,645,3,415,0.623490,-5.000000e-01,1720,-0.129866
2,3,357,11,388,0.623490,1.000000e+00,1589,-0.394460
3,10,700,28,346,0.623490,-1.836970e-16,210,-0.362179
4,13,686,20,115,-0.900969,-5.000000e-01,1135,0.551033


In [5]:
# TODO: Train the Linear model for predicting sales
class LinearModel(nn.Module):
    def __init__(self, input_dim: int, output_dim: int) -> None:
        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        self.elu = nn.ELU()
        self.dropout = nn.Dropout(p=0.3)

        # Weight Initialization
        init.kaiming_uniform_(self.linear.weight, nonlinearity='linear')

    def forward(self, x):
        x = self.elu(self.linear(x))
        x = self.dropout(x)
        x = self.elu(x)
        return x

# set random seed for reproducability
random.seed(42)
torch.manual_seed(42)
np.random.seed(42)

# TODO: Create the dataset class for training
class SalesDataset(Dataset):
    def __init__(self, data: pd.DataFrame) -> None:
        super().__init__()
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        features = torch.tensor(row[:-1].values, dtype=torch.float32)
        target = torch.tensor(row[-1], dtype=torch.float32)
        return features, target

# TODO: Create the dataset for training and validation
train_dataset = SalesDataset(train_data)
val_dataset = SalesDataset(val_data)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# TODO: Initialize the model, loss func and optimizer for a training loop of 5 epochs
input_dim = train_data.shape[1] - 1
output_dim = 1
model = LinearModel(input_dim, output_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.1)

metrics = torchmetrics.MeanAbsoluteError()
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# TODO: Training the model
for epoch in range(num_epochs):
    for i, (features, target) in enumerate(train_loader):
        features = features.to(device)
        target = target.to(device)

        optimizer.zero_grad()
        outputs = model(features)

        loss = criterion(outputs, target.view(-1, 1))
        mae = metrics(outputs, target.view(-1, 1))

        loss.backward()
        optimizer.step()
        if (i + 1) % 100 == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}, MAE: {mae.item():.4f}")

# TODO: Evaluate the model on the validation set
model.eval()
val_loss = 0.0
with torch.no_grad():
    for features, target in val_loader:
        features = features.to(device)
        target = target.to(device)

        outputs = model(features)
        loss = criterion(outputs, target.view(-1, 1))
        mae = metrics(outputs, target.view(-1, 1))
        val_loss += loss.item()
    val_loss /= len(val_loader)
print(f"Validation Loss: {val_loss:.4f}, MAE: {mae.item():.4f}")

Epoch [1/10], Step [100/221], Loss: 1.9988, MAE: 0.7370
Epoch [1/10], Step [200/221], Loss: 0.5967, MAE: 0.5792
Epoch [2/10], Step [100/221], Loss: 0.9105, MAE: 0.6350
Epoch [2/10], Step [200/221], Loss: 1.7289, MAE: 0.8149
Epoch [3/10], Step [100/221], Loss: 1.1534, MAE: 0.6400
Epoch [3/10], Step [200/221], Loss: 0.6158, MAE: 0.6073
Epoch [4/10], Step [100/221], Loss: 1.2604, MAE: 0.6923
Epoch [4/10], Step [200/221], Loss: 1.4457, MAE: 0.7997
Epoch [5/10], Step [100/221], Loss: 1.3793, MAE: 0.8038
Epoch [5/10], Step [200/221], Loss: 2.1161, MAE: 0.8009
Epoch [6/10], Step [100/221], Loss: 0.3293, MAE: 0.5181
Epoch [6/10], Step [200/221], Loss: 0.6788, MAE: 0.6287
Epoch [7/10], Step [100/221], Loss: 1.2167, MAE: 0.7752
Epoch [7/10], Step [200/221], Loss: 0.8031, MAE: 0.7184
Epoch [8/10], Step [100/221], Loss: 0.2142, MAE: 0.4122
Epoch [8/10], Step [200/221], Loss: 1.1306, MAE: 0.6531
Epoch [9/10], Step [100/221], Loss: 0.4598, MAE: 0.5307
Epoch [9/10], Step [200/221], Loss: 0.5343, MAE:

In [ ]:
# TODO: Create data sequences function for the train and test sets, to ensure that x, y are in the same order \
# y - should have two columns, for product-name and , sales as target

def create_dat_sequences(data: pd.DataFrame, seq_len: int = 20) -> Tuple:
    """This function creates the data sequences for the time series data"""
    data = data.copy()
    

In [ ]:
# TODO: Create a multi-output model for predicting sales and the product name
class MultiOutputModel(nn.Module):
    """This class implements a multi-input and multi-output model for predicting sales and product name."""
    def __init__(
            self,
            input_dim: int,
            output_dim: int,
            hidden_size: int = 64,
            num_layers: int = 2,
            dropout: float = 0.3
    ):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=True
        )

        self.elu = nn.ELU()
        self.linear = nn.Linear(hidden_size * 2, output_dim)
        self.batchnorm = nn.BatchNorm1d(output_dim)

        # Kaiming weight initialization
        nn.init.kaiming_uniform_(self.linear.weight, nonlinearity='linear')
        nn.init.zeros_(self.linear.bias)
        nn.init.zeros_(self.batchnorm.weight)
        nn.init.zeros_(self.batchnorm.bias)

        

SyntaxError: invalid syntax (353366794.py, line 8)